In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
%matplotlib qt

In [54]:
def find_CP(x,y,zerorange=900):
    """Finds the CP from F-Z curves by auto-thresholding data. 
    Algorithm writteb by Prof Massimo Vassalli and avilable
    at the open source repository: https://github.com/CellMechLab/softmech"""
    deg = 0 
    worky = np.copy(y)
    xtarget = np.min(x) + zerorange *1e-3
    jtarget = np.argmin( np.abs(x-xtarget) )

    #which direction?
    if x[0]<x[-1]:
        xlin = x[:jtarget]
        ylin = worky[:jtarget]
        m,q = np.polyfit(xlin,ylin,1)
    else:
        xlin = x[jtarget:]
        ylin = worky[jtarget:]
        m,q = np.polyfit(xlin,ylin,1)

    worky = worky-m*x-q

    differences = (worky[1:]+worky[:-1])/2
    midpoints = np.array(list(set(differences)))
    midpoints.sort()

    crossings = []
    for threshold in midpoints[midpoints>0]:
        crossings.append( np.sum( np.bitwise_and( (worky[1:]>threshold),(worky[:-1]<threshold) )))
    crossings=np.array(crossings)

    inflection = midpoints[midpoints>0][np.where(crossings==1)[0][0]]
    jcpguess = np.argmin( np.abs(differences-inflection) )+1

    xcp = x[jcpguess]
    ycp = y[jcpguess]
    return [xcp, ycp]

def average_curve(arrs):
    """Finds the average array from a list of arrays containing 
    arrays of different lengths."""
    lens = [len(i) for i in arrs]
    arr = np.ma.empty((np.max(lens),len(arrs)))
    arr.mask = True
    for idx, l in enumerate(arrs):
        arr[:len(l),idx] = l
    return arr.mean(axis = -1), arr.std(axis=-1)

In [70]:

filedir = "/Users/giuseppeciccone/Library/CloudStorage/OneDrive-UniversityofGlasgow/PhD/PyModules/stretching/data/K.xlsx"
file = pd.ExcelFile(filedir)
sheet_names=file.sheet_names

In [71]:
fig,axs = plt.subplots(1,3,figsize=(15,5))
all_loads = []
all_pos = []
for i, name in enumerate(sheet_names):
    data= pd.read_excel(filedir,sheet_name=name)
    pos = np.array(data.loc[:,"POSITION"])
    load = np.array(data.loc[:,"LOADCELL"])
    x0,y0 = find_CP(pos,load,zerorange=3000)
    x0ind = np.argmin((pos-x0)**2)
    y0ind = np.argmin((load-y0)**2)
    rupture = max(load)
    irupture = np.argmin((load-rupture)**2)
    if rupture<0.02: #avoid bad curves
        axs[0].plot(pos,load,color='black',lw=1)
        axs[0].plot(pos[y0ind:irupture],load[y0ind:irupture],'o',color='red',alpha=0.1)
        #align curves to the same origin
        pos = pos - x0
        load = load - y0
        axs[1].plot(pos[y0ind:irupture],load[y0ind:irupture],color='black',lw=1)
        all_loads.append(load[y0ind:irupture])
        all_pos.append(pos[y0ind:irupture])
#average curves
avg_force,avg_force_std = average_curve(all_loads)
avg_pos,avg_pos_std = average_curve(all_pos)
axs[2].plot(avg_pos,avg_force)
axs[2].fill_between(avg_pos,avg_force-avg_force_std/2.0,avg_force+avg_force_std/2.0,alpha=0.5)
plt.show()